<a href="https://colab.research.google.com/github/EnayatAria/UAV-image-classification-by-CNN/blob/main/CNN_vineyard_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This part is for debugging the code
%xmode Verbose
%pdb on
from IPython.core.debugger import set_trace

Exception reporting mode: Verbose
Automatic pdb calling has been turned ON


In [ ]:
!pip install spectral

     |████████████████████████████████| 212 kB 5.4 MB/s 


In [ ]:
from scipy import io
import numpy as np
import spectral.io.envi as envi
import pandas as pd
import numpy as np
import torch
import torch.utils.data as dataf
import torch.nn as nn
import matplotlib.pyplot as plt


In [ ]:
# reading the image data and ground truth
dataset='Sinkhorn40000_TS'
scene='subset1'
# for lambda_reg in range(40000, 40001):
OV_type=['EMD', 'Sinkhorn','L1L2','Laplace']
# OV_type=['Laplace']

dir_list=[ 'without Grass transformation using classification map', 'with Grass transformation using training site',  
          'with Grass transformation using classification map','without Grass transformation using training site']

# for iii in range (40000, 1000000, 50000):

for dir in dir_list:
  '''
  if dir == 'without Grass transformation using training site':
    ov_t = 'ts'
    # lambda_reg=340000
  else:
    ov_t = 'cm'
    # lambda_reg=940000
    '''
  reg = 1000
  while reg<=1000000:
  # for ov_t in OV_type:
    for iii in range (40000, 1000000, 200000):
      dataset='L1L2_lambda_'+str(iii)+'_regGS_'+str(reg) 
      #DataPath = '/content/drive/MyDrive/Gaillac-UAV-ML-2016/subset'
      #txtPath='/content/drive/MyDrive/Gaillac-UAV-ML-2016/roi_txt.txt'
      #DataPath = '/content/drive/MyDrive/dataset/Minervois/transformed_sh'
      DataPath_target ='/content/drive/MyDrive/dataset/Minervois/'+scene+'/'+ dir + '/REG_GS/'+dataset
      #txtPath='/content/drive/MyDrive/dataset/Minervois/ENVI/classes.txt'
      # savepath ='/content/drive/MyDrive/Gaillac-UAV-ML-2016/2DCNN-14.mat'

      # Reading envi dataset
      img =envi.open(DataPath_target+'.hdr', DataPath_target+'.img')
      Data=img.load()
      #Data=Data[1050:1550,800:1300,:]
      #Data = Data.astype(np.float32)

      # some initialization
      patchsize = 16  # input spatial size for 2D-CNN
      batchsize = 128  # select from [16, 32, 64, 128], the best is 64
      EPOCH = 200
      LR = 0.001

      # without dimensionality reduction
      pad_width = np.floor(patchsize / 2)
      pad_width = np.int(pad_width)

      # standardization method 2: map to zero mean and one std
      [m, n, l] = np.shape(Data)
      for i in range(l):
          mean = np.mean(Data[:, :, i])
          std = np.std(Data[:, :, i])
          Data[:, :, i] = (Data[:, :, i] - mean) / std

      Classes=4

      x = Data
      # boundary interpolation
      temp = x[:, :, 0]
      temp=np.squeeze(temp)
      temp2 = np.pad(temp, pad_width, 'symmetric')
      [m2, n2] = temp2.shape
      x2 = np.empty((m2, n2, l), dtype='float32')
      for i in range(l):
          temp = np.squeeze(x[:, :, i])
          pad_width = np.floor(patchsize / 2)
          pad_width = np.int(pad_width)
          temp2 = np.pad(temp, pad_width, 'symmetric')
          x2[:, :, i] = temp2

      # to check if cuda is available or not
      # when the GPU is on from the 'Runtime' tab, the result would be 'True'
      torch.cuda.is_available()

      # construct the network
      class CNN(nn.Module): # nn.Module is the base class for all neural network modules
          def __init__(self):
              super(CNN, self).__init__()  # super() allows us to access methods in the base class
              self.conv1 = nn.Sequential(  # a sequential container
                  nn.Conv2d(
                      in_channels=l,
                      out_channels=OutChannel,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                  ),
                  nn.BatchNorm2d(OutChannel),
                  nn.ReLU(),
                  nn.MaxPool2d(kernel_size=2),
              )
              self.conv2 = nn.Sequential(
                  nn.Conv2d(OutChannel, OutChannel * 2, 3, 1, 1),
                  nn.BatchNorm2d(OutChannel * 2),
                  nn.ReLU(),
                  nn.MaxPool2d(2),
                  # nn.Dropout(0.5),

              )
              self.conv3 = nn.Sequential(
                  nn.Conv2d(OutChannel * 2, OutChannel * 4, 3, 1, 1),
                  nn.BatchNorm2d(OutChannel * 4),
                  nn.ReLU(),
                  nn.AdaptiveMaxPool2d(1),
                  # nn.Dropout(0.5),

              )

              self.out = nn.Linear(OutChannel * 4, Classes)  # fully connected layer, output 16 classes
          
          def forward(self, x):
              #set_trace()
              x = self.conv1(x)
              x = self.conv2(x)
              x = self.conv3(x)
              x = x.view(x.size(0), -1)  # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
              output = self.out(x)
              return output

      OutChannel = 32
      cnn = CNN()
      print('The structure of the designed network', cnn)

      # move model to GPU
      cnn.cuda()
      optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)  # optimize all cnn parameters
      loss_func = nn.CrossEntropyLoss()  # the target label is not one-hotted
      BestAcc = 0

      # show the whole image
      # The whole data is too big to test in one time; So dividing it into several parts

      cnn.load_state_dict(torch.load('/content/drive/MyDrive/Gaillac-UAV-ML-2016/net_params_2DCNN2.pkl'))#, map_location=torch.device('cpu')))

      part = 5000
      pred_all = np.empty((m*n, 1), dtype='float32')

      number = m*n//part
      for i in range(number):
          D = np.empty((part, l, patchsize, patchsize), dtype='float32')
          count = 0
          for j in range(i*part, (i+1)*part):
              row = j//n
              col = j - row*n
              row2 = row + pad_width
              col2 = col + pad_width
              patch = x2[(row2 - pad_width):(row2 + pad_width), (col2 - pad_width):(col2 + pad_width), :]
              patch = np.reshape(patch, (patchsize * patchsize, l))
              patch = np.transpose(patch)
              patch = np.reshape(patch, (l, patchsize, patchsize))
              D[count, :, :, :] = patch
              count += 1

          temp = torch.from_numpy(D)
          temp = temp.cuda()
          temp2 = cnn(temp)
          temp3 = torch.max(temp2, 1)[1].squeeze()
          pred_all[i*part:(i+1)*part, 0] = temp3.cpu()
          del temp, temp2, temp3, D

      if (i+1)*part < m*n:
          D = np.empty((m*n-(i+1)*part, l, patchsize, patchsize), dtype='float32')
          count = 0
          for j in range((i+1)*part, m*n):
              row = j // n
              col = j - row * n
              row2 = row + pad_width
              col2 = col + pad_width
              patch = x2[(row2 - pad_width):(row2 + pad_width), (col2 - pad_width):(col2 + pad_width), :]
              patch = np.reshape(patch, (patchsize * patchsize, l))
              patch = np.transpose(patch)
              patch = np.reshape(patch, (l, patchsize, patchsize))
              D[count, :, :, :] = patch
              count += 1

          temp = torch.from_numpy(D)
          temp = temp.cuda()
          temp2 = cnn(temp)
          temp3 = torch.max(temp2, 1)[1].squeeze()
          pred_all[(i + 1) * part:m*n, 0] = temp3.cpu()
          del temp, temp2, temp3, D


      pred_all = np.reshape(pred_all, (m, n)) + 1
      '''
      OA = OA.numpy()
      pred_y = pred_y.cpu()
      pred_y = pred_y.numpy()
      TestDataLabel = TestLabel.cpu()
      TestDataLabel = TestDataLabel.numpy()

      io.savemat(savepath, {'PredAll': pred_all, 'OA': OA, 'TestPre': pred_y, 'TestLabel': TestDataLabel})
      '''
      # print io.loadmat(savepath)
      #

      plt.figure()
      plt.imshow(pred_all)
      plt.show()

      import matplotlib.pyplot as plt
      from matplotlib import cm
      from matplotlib.colors import ListedColormap, LinearSegmentedColormap

      #  matplotlib color specification: https://matplotlib.org/stable/tutorials/colors/colors.html
      cmap = ListedColormap(["red", "lightgreen", "darkgreen", "yellow"])
      #plot_examples([cmap])


      plt.figure()
      plt.imshow(pred_all, cmap=cmap)
      plt.show()

      import matplotlib.pyplot as plt
      from datetime import datetime
      now = datetime.now()
      dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
      md=img.metadata
      #set_trace()
      md['description'] = ' classification '+dataset+' transformed using training site '+dt_string
      del md['band names']
      class_names=['Unclassified', 'Soil', 'Vine', 'Shadow', 'Grass']
      class_colors=[(0,   0,   0), (254,   0,   0), (144, 238, 144), (0, 100,   1), (255, 255,   1)]
      #img = envi.create_image('savemappath', md)
      #set_trace()
      envi.save_classification(DataPath_target+'_classification.hdr' , 
                              pred_all, dtype=np.float32, metadata=md, force=True, class_names=class_names, class_colors=class_colors)
    reg*=10
      #savemappath='/content/drive/MyDrive/Gaillac-UAV-ML-2016/ClassifiedMap.hdr'

In [ ]:
DataPath_target

'/content/drive/MyDrive/dataset/Minervois/subset1/best_classification_res(reg_e_chnage)/Sinkhorn40000_TS'

In [ ]:
print(np.mean(np.mean(Data, axis=0), axis=0))

[-3.9309263e-08  3.2398106e-07 -8.4236262e-08  2.0390749e-07
 -2.1898747e-07]
